# Walkability analysis notebook

This notebook takes the data objects saved in `data/prepared/` and produced by `data_ingestion.ipynb`, and operates on them to create measures of walkability by geography.

This is achieved by computing the shortest walking distance to different categories of place, and averaging it across each geography.

This requires trips to be generated and analysed:

1. Constructing the starting points. We distribute starting points along the network nodes with a density proportional to the local population (at SA1 granularity)
2. Finding the end points. From each starting point, we want to obtain a sample of end points from point-of-interest data that is within some walking distance (e.g. 5km)
3. Constructing the trips. Each trip is a pair of a start point and the local end points, which are mapped to the geographically closest network node.
4. Compute the distance of all trips on the graph, using the edge weights, and take the minimum per category of destination, and the number within the search radius. Network libraries perform this efficiently.
5. This minimum distance and count per category are properties of the starting location. Statistics per geographic area can be computed by averaging them across the starting locations in the geography.
6. The walkability measures can be used in combination with other census data to investigate causal factors and correlates of walkability



In [ ]:
# imports, with everything converted to meters for distances

# libraries

import pandas as pd
import geopandas as gpd
import numpy as np
import pandana as pdna
import itertools
from shapely.geometry import Point
from functools import reduce


# point of interest geodataframes

coffee_gdf = gpd.read_feather('data/prepared/coffee.feather').to_crs("EPSG:7855") # everywhere you can get a coffee in Melbourne
places_gdf = gpd.read_feather('data/prepared/places.feather').to_crs("EPSG:7855") # general places of interest

# geography geodataframes

lga_gdf = gpd.read_feather('data/prepared/lga.feather').to_crs("EPSG:7855") # local gov areas
poa_gdf = gpd.read_feather('data/prepared/poa.feather').to_crs("EPSG:7855") # postcodes
sal_gdf = gpd.read_feather('data/prepared/sal.feather').to_crs("EPSG:7855") # suburbs
sa1_gdf = gpd.read_feather('data/prepared/sa1.feather').to_crs("EPSG:7855") # SA1 statistical areas

# road network data

edges = pd.read_feather('data/prepared/graph_edges.feather')
nodes = pd.read_feather('data/prepared/graph_nodes.feather')

# construct pandana network object

network_definition = {
    'node_x': nodes['x'],
    'node_y': nodes['y'],
    'edge_from': edges['u'],
    'edge_to': edges['v'],
    'edge_weights': edges[['w']], # length in metres
    'twoway': True
}

graph = pdna.Network(**network_definition)


## Construction of starting points

In [ ]:
print(f'{sa1_gdf.population.sum():,} people live in the SA1s of interest, which will be distributed across {len(nodes):,} nodes ({sa1_gdf.population.sum()/len(nodes):,.2} persons per node)')

3,921,481 people live in the SA1s of interest, which will be distributed across 1,689,061 nodes (2.3 persons per node)


In [ ]:
nodes_df = graph.nodes_df

nodes_df['geometry'] = nodes_df.apply(lambda row: Point(row['x'],row['y']), axis=1)
nodes_gdf = gpd.GeoDataFrame(nodes_df,geometry='geometry', crs="EPSG:7855").reset_index()

# attach population numbers
nodes_gdf = gpd.sjoin(nodes_gdf,sa1_gdf[['population','geometry','geography_name']], how='left')

# compute weights per node (how many people each node represents)
node_counts_per_sl1 = nodes_gdf.groupby('geography_name').size().reset_index(name='node_count')
nodes_gdf = nodes_gdf.merge(node_counts_per_sl1, on='geography_name', how='left')
nodes_gdf['node_weight'] = nodes_gdf['population'] / nodes_gdf['node_count'].replace(0, 1)

/var/folders/df/p5245drj1ks2x4rkg5fmc4_40000gn/T/ipykernel_84268/3006491564.py:7: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:7844
Right CRS: EPSG:7855

  nodes_gdf = gpd.sjoin(nodes_gdf,sa1_gdf[['population','geometry','geography_name']], how='left')


## Construction of destination points

In [40]:

# returns the same gdf with the closest node ids in a column (if there's one close enough)
def find_nearest_nodes(poi_gdf, graph, max_dist_m=30):
    # Create a copy of the poi_gdf to avoid modifying the original dataframe
    poi_gdf = poi_gdf.copy()

    # Extract x and y coordinates
    xs = poi_gdf.geometry.x.values
    ys = poi_gdf.geometry.y.values

    # Find the nearest nodes
    nearest_nodes = graph.get_node_ids(xs, ys, mapping_distance=max_dist_m)
    poi_gdf['nearest_node'] = nearest_nodes

    # Calculate unmatched locations
    unmatched = poi_gdf['nearest_node'].isna().sum()
    print(f'{unmatched} locations were not able to be matched within {max_dist_m}m')

    # Filter out unmatched locations
    matched_nodes = poi_gdf.dropna(subset=['nearest_node']).copy()

    for node_id in matched_nodes['nearest_node'].unique():
        matched_nodes.loc[matched_nodes['nearest_node'] == node_id, 'node_x'] = graph.nodes_df.loc[node_id].x
        matched_nodes.loc[matched_nodes['nearest_node'] == node_id, 'node_y'] = graph.nodes_df.loc[node_id].y

    return matched_nodes

# Apply the function to the GeoDataFrames
place_nodes_gdf = find_nearest_nodes(places_gdf, graph)
coffee_nodes_gdf = find_nearest_nodes(coffee_gdf, graph)


coffee_nodes_gdf


7580 locations were not able to be matched within 30m
374 locations were not able to be matched within 30m


,name,category,source,geometry,nearest_node,node_x,node_y
0,North Point Cafe,coffee_available,here_api,POINT (323029.685 5803637.676),5.713212e+08,323045.021224,5.803636e+06
1,Ladygreen,coffee_available,here_api,POINT (323691.895 5803852.887),7.403699e+08,323694.027992,5.803872e+06
2,Superrandom,coffee_available,here_api,POINT (323688.855 5803830.618),1.110030e+10,323688.246534,5.803827e+06
3,The Little Ox,coffee_available,here_api,POINT (323715.194 5804037.679),9.976415e+09,323713.761008,5.804029e+06
4,Saint Martins Cafe,coffee_available,here_api,POINT (324355.613 5803711.709),1.109813e+10,324365.498083,5.803710e+06
...,...,...,...,...,...,...,...
4828,Crafty Cafe,coffee_available,here_api,POINT (315917.835 5837023.927),3.218886e+09,315911.566771,5.837027e+06
4829,Max Brenner,coffee_available,here_api,POINT (315767.171 5837046.113),1.028948e+10,315779.341584,5.837032e+06
4830,THE COFFEE CLUB,coffee_available,here_api,POINT (315854.779 5837237.901),5.936035e+09,315849.322112,5.837258e+06
4831,Donut King,coffee_available,here_api,POINT (315704.033 5837104.659),1.028948e+10,315701.919566,5.837109e+06


## Calculating distances on the graph

In [131]:
max_distance = 2000
max_items = 50

destination_nodes_gdf = pd.concat([
    place_nodes_gdf[['name','category','node_x','node_y','nearest_node']],
    coffee_nodes_gdf[['name','category','node_x','node_y','nearest_node']]
])

categories = destination_nodes_gdf['category'].unique()

for category in categories:
    filtered_destinations = destination_nodes_gdf[destination_nodes_gdf['category']==category]
    graph.set_pois(category,max_distance,max_items,filtered_destinations['node_x'],filtered_destinations['node_y'])

graph.poi_category_names


['restaurant',
 'grocery or supermarket',
 'cafe',
 'bar or pub',
 'place of worship',
 'tourist attraction',
 'community area',
 'aged care',
 'park area',
 'school',
 'child care',
 'library',
 'emergency services',
 'medical facility',
 'entertainment centre',
 'swimming pool',
 'tertiary institution',
 'art gallery',
 'museum',
 'coffee_available']

In [132]:

# interested in returning, per category, how close is the closest, and how many are within 500m, 1km, 2km (up to a max of 100)

def summarise_distances_per_node(category):
    df = graph.nearest_pois(distance=max_distance, category=category, num_pois=max_items)
    df_out = df.copy()[[1]]
    df_out[category + ' - within 2km'] = (df < 2000).sum(axis=1)
    df_out[category + ' - within 1km'] = (df < 1000).sum(axis=1)
    df_out[category + ' - within 500m'] = (df < 500).sum(axis=1)
    df_out[category + ' - closest'] = df.min(axis=1).replace(max_distance, np.nan)
    return df_out.drop(columns=[1])


summaries = []

for category in categories:
    summaries.append(summarise_distances_per_node(category))
    print(f'{len(summaries)} of {len(categories)} done')

distances_df = reduce(lambda left, right: pd.merge(left, right, on='osmid'), summaries)


1 of 20 done
2 of 20 done
3 of 20 done
4 of 20 done
5 of 20 done
6 of 20 done
7 of 20 done
8 of 20 done
9 of 20 done
10 of 20 done
11 of 20 done
12 of 20 done
13 of 20 done
14 of 20 done
15 of 20 done
16 of 20 done
17 of 20 done
18 of 20 done
19 of 20 done
20 of 20 done


In [133]:
distances_df

,restaurant - within 2km,restaurant - within 1km,restaurant - within 500m,restaurant - closest,grocery or supermarket - within 2km,grocery or supermarket - within 1km,grocery or supermarket - within 500m,grocery or supermarket - closest,cafe - within 2km,cafe - within 1km,...,art gallery - within 500m,art gallery - closest,museum - within 2km,museum - within 1km,museum - within 500m,museum - closest,coffee_available - within 2km,coffee_available - within 1km,coffee_available - within 500m,coffee_available - closest
osmid,,,,,,,,,,,,,,,,,,,,,
579259,50,18,3,345.002014,15,5,1,442.201996,2,1,...,0,NaN,0,0,0,NaN,23,9,3,345.002014
579260,43,1,0,759.739990,11,1,0,887.843994,2,0,...,0,NaN,0,0,0,NaN,16,1,0,759.739990
579261,48,6,4,0.000000,6,2,1,421.539001,3,1,...,0,NaN,0,0,0,NaN,17,3,2,0.000000
579265,32,6,0,583.995972,7,1,0,752.965027,2,1,...,0,NaN,0,0,0,NaN,13,3,0,583.995972
579267,50,22,1,68.067001,4,0,0,1240.296021,3,2,...,0,NaN,0,0,0,NaN,17,4,0,797.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276470586,39,22,20,220.656998,8,5,3,255.046997,7,7,...,0,NaN,0,0,0,NaN,19,12,9,255.046997
11276470587,39,22,20,215.488998,8,5,3,249.878998,7,7,...,0,NaN,0,0,0,NaN,19,12,9,249.878998
11276470588,39,22,20,206.177002,8,5,3,240.567001,7,7,...,0,NaN,0,0,0,NaN,19,12,9,240.567001


## Aggregate across different geographies

In [139]:
# we need to join the distances df to the nodes gdf to get the geometries and weights, and then spatial join to enable aggregation

# start with suburbs, then genericise


final_nodes_gdf = nodes_gdf.merge(distances_df.reset_index(),how='left',on=['osmid'])

final_nodes_gdf.to_feather('data/final/final_nodes.feather')

In [144]:
final_nodes_gdf

,osmid,x,y,geometry,index_right,population,geography_name,node_count,node_weight,restaurant - within 2km,...,art gallery - within 500m,art gallery - closest,museum - within 2km,museum - within 1km,museum - within 500m,museum - closest,coffee_available - within 2km,coffee_available - within 1km,coffee_available - within 500m,coffee_available - closest
0,579259,335501.287752,5.800198e+06,POINT (335501.28775 5800197.76430),6398,425,21204131015,63,6.746032,50,...,0,NaN,0,0,0,NaN,23,9,3,345.002014
1,579260,335384.338411,5.799317e+06,POINT (335384.33841 5799316.96717),6402,617,21204131019,325,1.898462,43,...,0,NaN,0,0,0,NaN,16,1,0,759.739990
2,579261,336519.448334,5.799162e+06,POINT (336519.44833 5799161.68831),6394,1157,21204131011,203,5.699507,48,...,0,NaN,0,0,0,NaN,17,3,2,0.000000
3,579265,336613.589061,5.799543e+06,POINT (336613.58906 5799542.63209),6544,0,21204131733,170,0.000000,32,...,0,NaN,0,0,0,NaN,13,3,0,583.995972
4,579267,336507.598142,5.801855e+06,POINT (336507.59814 5801855.34957),7163,1166,21205156702,2970,0.392593,50,...,0,NaN,0,0,0,NaN,17,4,0,797.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689056,11276470586,314088.029452,5.823816e+06,POINT (314088.02945 5823816.20637),4535,13,21001122701,3241,0.004011,39,...,0,NaN,0,0,0,NaN,19,12,9,255.046997
1689057,11276470587,314089.840541,5.823811e+06,POINT (314089.84054 5823811.37330),4535,13,21001122701,3241,0.004011,39,...,0,NaN,0,0,0,NaN,19,12,9,249.878998
1689058,11276470588,314093.420333,5.823803e+06,POINT (314093.42033 5823802.80533),4535,13,21001122701,3241,0.004011,39,...,0,NaN,0,0,0,NaN,19,12,9,240.567001
1689059,11276470589,314096.489426,5.823798e+06,POINT (314096.48943 5823797.72305),4535,13,21001122701,3241,0.004011,39,...,0,NaN,0,0,0,NaN,19,12,9,234.625000


In [138]:
sal_gdf

,link_key,area_sqkm,geography_name,geometry,population,geography,median_age,median_mortgage_repayment_monthly,median_rent_weekly,average_persons_per_bedroom,...,dwelling_state_housing,dwelling_community_housing,dwelling_rented,pct_dwellings_unoccupied,pct_households_wo_cars,pct_renters,pct_owner_occupiers,pct_apartments,pct_houses,pct_townhouses
0,20002,1.7405,Abbotsford (Vic.),"POLYGON ((324084.391 5814866.896, 324080.091 5...",9088,SAL,33,2167,425,1.0,...,69,60,2436,0.186516,0.213137,0.553888,0.424511,0.627558,0.102092,0.238745
1,20003,1.5515,Aberfeldie,"POLYGON ((314655.130 5818158.052, 314630.100 5...",3925,SAL,41,2600,440,0.8,...,25,3,279,0.080189,0.035216,0.203501,0.771699,0.139314,0.666667,0.183807
2,20011,6.7302,Aintree,"POLYGON ((294817.955 5822002.822, 294680.391 5...",7982,SAL,30,2167,420,0.9,...,0,0,446,0.033095,0.013102,0.206100,0.780037,0.000000,0.989834,0.009242
3,20015,3.6748,Airport West,"POLYGON ((313123.862 5823617.332, 313155.748 5...",8173,SAL,39,2085,401,0.8,...,11,0,962,0.091557,0.067313,0.279814,0.697789,0.016579,0.625364,0.354276
4,20017,1.8634,Albanvale,"POLYGON ((303447.345 5820598.232, 303590.131 5...",5641,SAL,37,1500,325,0.9,...,54,9,421,0.050000,0.061339,0.232983,0.730493,0.017709,0.960708,0.022136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,22892,2.9327,Yallambie,"POLYGON ((332621.046 5822768.746, 332544.517 5...",4161,SAL,35,2167,340,0.8,...,28,3,343,0.053610,0.025974,0.259652,0.718395,0.002271,0.890235,0.104466
345,22898,28.3934,Yan Yean,"POLYGON ((337579.598 5841471.686, 337488.266 5...",246,SAL,48,2550,388,0.7,...,0,0,7,0.119565,0.000000,0.086420,0.728395,0.000000,1.000000,0.000000
346,22916,15.3453,Yarrambat,"POLYGON ((338205.798 5834239.374, 338218.742 5...",1602,SAL,47,2600,450,0.8,...,0,0,21,0.069098,0.012500,0.043299,0.925773,0.000000,1.002062,0.000000
347,22917,5.6587,Yarraville,"POLYGON ((314247.616 5813194.037, 314251.860 5...",15636,SAL,37,2500,462,0.9,...,96,37,2011,0.084930,0.094723,0.331848,0.648680,0.109076,0.717162,0.161881


In [140]:
sal_joined_gpd = gpd.sjoin(final_nodes_gdf,sal_gdf)

sal_joined_gpd


/var/folders/df/p5245drj1ks2x4rkg5fmc4_40000gn/T/ipykernel_84268/3417953526.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:7844
Right CRS: EPSG:7855

  gpd.sjoin(final_nodes_gdf,sal_gdf)


ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined